In [1]:
from Brokers.Alpaca.Bots.Alpaca_keyes import API_KEY, SECRET_KEY
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

In [2]:
api_key = API_KEY
secret_key = SECRET_KEY

symbol = "TSLA"

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key)

In [3]:
now = datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 15, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 50),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    #limit = 2,                                               # specify limit
)

In [ ]:
# timestamp is in UTC time
stock_historical_data_client.get_stock_bars(req).df

I have 15 min to calculate the missing candle, I need to write a function which converts live datasream into a 15 min candle

In [5]:
import nest_asyncio
nest_asyncio.apply()

async def stock_data_stream_handler(data):
    print(data)

In [6]:
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

In [ ]:
import asyncio
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

# Subscribe to quotes and trades
stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

async def run_stream():
    await stock_data_stream_client.run()

async def main():
    # Create a task for running the stream
    stream_task = asyncio.create_task(run_stream())

    # Let the stream run for a specific duration (e.g., 60 seconds)
    await asyncio.sleep(10)

    # Stop the stream client
    await stock_data_stream_client.close() # do I need close() or stop()?

    # Wait for the stream task to complete
    await stream_task

# Run the main function
asyncio.run(main())
